In [1]:
#Imports
import sys,glob,os
import numpy as np
import cPickle as pickle
sys.path.append(r'C:\Data\Alistair\code\python-functions\python-functions-library')
import TadImagingAnalysis as tia
import matplotlib.pylab as plt

In [210]:
master_folder = r'D:\Bogdan\HCT16_untreated_noAB-2col_6_29_2018'
analysis_folder = master_folder+os.sep+r'Analysis2'

In [214]:
files = glob.glob(analysis_folder+os.sep+'*_cand.pkl')
cy7ptsf,cy5ptsf=[],[]
for fl in files[::-1]:
    dic = pickle.load(open(fl,'rb'))

    chroms = dic.keys()
    chrom = chroms[0]
    #cy5k,cy7k = 'H3R5','H43P5'
    for cy5k,cy7k in [['H1R1','H1R2'],['H2R3','H2R4'],['H3R5','H3R6'],['H4R7','H4R8']]:
    #cy5k,cy7k = 'H1R1','H1R2'

        cy5pts,cy7pts = [],[]
        for chrom in chroms:
            dic_ = dic[chrom][0]
            if dic_.has_key(cy5k):
                cy5pts.extend(dic_[cy5k][:,1:4])
            if dic_.has_key(cy7k):
                cy7pts.extend(dic_[cy7k][:,1:4])
        cy7pts = np.array(cy7pts)
        cy5pts = np.array(cy5pts)

        cutoff=2
        M = cdist(cy5pts,cy7pts)
        longdim = np.argmax(M.shape)
        inds_sh = np.argmin(M,longdim)
        dists_sh = np.min(M,longdim)
        keep = dists_sh<cutoff
        ind1,ind2 = inds_sh[keep],np.arange(len(inds_sh))[keep]
        if longdim==1:
            ind1,ind2 = ind2,ind1
        cy5ptsf.extend(cy5pts[ind1])
        cy7ptsf.extend(cy7pts[ind2])
cy5ptsf = np.array(cy5ptsf)
cy7ptsf = np.array(cy7ptsf)

In [215]:
cy5ptsf = np.array(cy5ptsf)
cy7ptsf = np.array(cy7ptsf)
dif = cy5ptsf-cy7ptsf
x,y=median_vec(cy5ptsf[:,1],dif[:,1],res=10)
plt.plot(x,y,'o',alpha=0.2)
#plt.plot(dif[:,1])
#plt.plot(dif[:,2])
plt.show()

In [105]:
def median_vec(vecx,vecy,res=1,func=np.nanmedian):
    edges = np.arange(min(vecx),max(vecx),res)
    vecy_ = [func(vecy[(vecx>edges[i])&(vecx<=edges[i+1])]) for i in range(len(edges)-1)]
    vecx_ = [func(vecx[(vecx>edges[i])&(vecx<=edges[i+1])]) for i in range(len(edges)-1)]
    return vecx_,vecy_
    

In [216]:
def calc_color_matrix(x,y):
    """This gives a quadratic color transformation (in matrix form)
    x is Nx3 vector of positions in the reference channel (typically cy5)
    y is the Nx3 vector of positions in another channel (i.e. cy7)
    return m_ a 3x7 matrix which when multipled with x,x**2,1 returns y-x
    This m_ is indended to be used with apply_colorcor
    """ 
    x_ = np.array(x)
    y_ = np.array(y)-x_
    A = np.concatenate([x_[:],x_[:]**2,np.ones([len(x_),1])],axis=-1)
    m_ = [np.linalg.lstsq(A, y_[:,iy])[0] for iy in range(len(x_[0]))]
    m_=np.array(m_)
    return m_
def apply_colorcor(x,m_):
    """This applies chromatic abberation correction to order 2
    x is a Nx3 vector of positions
    m_ is a matrix computed by function calc_color_matrix
    y is the corrected vector in another channel"""
    x_ = np.array(x)
    A = np.concatenate([x_[:],x_[:]**2,np.ones([len(x_),1])],axis=-1)
    diff = [np.dot(A,m) for m in m_]
    return x_+np.array(diff).T
m_ = calc_color_matrix(cy5ptsf,cy7ptsf)
new_cy5ptsf = apply_colorcor(cy5ptsf,calc_color_matrix(cy5ptsf,cy7ptsf))
print map(list,m_)

[[0.025711432432609635, 0.00025512759143988401, 5.5686637638975243e-06, -0.00042340795417944021, -2.0436776461688556e-07, -1.5928705456769738e-08, 0.27953709196490806], [-0.00353052918100763, -0.00034442564277036681, -1.7870917577209923e-05, 6.3843139463714616e-05, 1.7033892711492658e-08, 2.3814093085910767e-08, -0.016009480066035722], [-0.016349292644475905, 6.5813576351769304e-05, -0.00048380735880232585, 0.00037854567020854658, -5.6600463991897216e-08, 1.4513584741887027e-07, 0.19207772922465599]]


In [205]:
print np.mean(cy5ptsf-cy7ptsf,0)*[250,150,150],np.mean(new_cy5ptsf-cy7ptsf,0)*[250,150,150]
print np.std(cy5ptsf-cy7ptsf,0)*[250,150,150],np.std(new_cy5ptsf-cy7ptsf,0)*[250,150,150]

[-168.79466563   38.29954531   28.57541054] [  6.35411606e-13  -3.37187071e-12  -5.78829572e-13]
[ 81.44561702  49.52776173  48.73766511] [ 79.91033877  46.08492502  45.18598552]


In [179]:
dims = [35,1000,1000]
z_,x_,y_ = np.indices(dims)
zxy = np.array(map(np.ravel,[z_,x_,y_])).T
diffmat = [np.dot(np.concatenate([zxy[:],zxy[:]**2,np.ones([len(zxy),1])],axis=-1),m) for m in m_]
diffmat = np.reshape(diffmat,[3]+dims)
plt.imshow(diffmat[0,15],interpolation='nearest')
plt.colorbar()
plt.show()

In [181]:
plt.imshow(diffmat[2,15],interpolation='nearest')
plt.colorbar()
plt.show()

In [95]:
np.mean(cy5ptsf-cy7ptsf,0)*[250,150,150]

array([-168.79466563,   38.29954531,   28.57541054])

In [84]:
plt.plot(

array([[-0.66238708,  0.69301646,  0.05749832],
       [-0.29503979,  0.6043958 , -0.04318648],
       [-0.08882304,  0.41995526, -0.15581835],
       ..., 
       [-0.76357645,  0.14577748,  0.21598387],
       [-0.88858803,  0.62521053,  0.25025893],
       [-0.54191139,  0.23519723,  0.42502168]])

In [39]:
from scipy.spatial.distance import cdist
plt.imshow(cdist(cy5pts,cy7pts)<3)
plt.show()

In [59]:
plt.plot(cy5pts[ind1,0],cy5pts[ind1,2],'o')
plt.plot(cy7pts[ind2,0],cy7pts[ind2,2],'o')
plt.show()